# Notebook 01: OOP Aplicada a AI/ML Engineering

**Clase extra de Python - Notebook 4 de la serie**

---

## Objetivos de aprendizaje

Al finalizar este notebook seras capaz de:

1. **Definir clases** con estado (atributos) y comportamiento (metodos) en contextos de AI/ML.
2. **Proteger invariantes** con encapsulamiento (`@property`, convenciones `_private`).
3. **Elegir entre herencia y composicion** segun el problema, aplicando polimorfismo.
4. **Usar `dataclasses`** para datos estructurados sin boilerplate.
5. **Aplicar logging** dentro de clases para trazabilidad de pipelines.
6. **Disenar APIs limpias** usando criterios de responsabilidad unica y superficie minima.

## Informacion del curso

| Item | Detalle |
|---|---|
| **Tiempo estimado** | 90 - 120 minutos |
| **Prerequisitos** | Notebooks 01-03 (variables, control de flujo, funciones) |
| **Dependencias** | Solo biblioteca estandar de Python (no se necesita `pip install`) |
| **Contexto** | AI Engineering - wrappers de modelos, servicios de inferencia, orquestadores |

---

In [ ]:
# =============================================================================
# SETUP - Imports y configuracion
# =============================================================================

from dataclasses import dataclass, field
import logging
from typing import Protocol, Optional, List, Dict, Any
from abc import ABC, abstractmethod
import time
import json

# Configuracion basica de logging para todo el notebook
logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s | %(name)-25s | %(levelname)-8s | %(message)s",
    datefmt="%H:%M:%S",
    force=True  # force=True para que funcione bien en Jupyter
)

# Verificacion rapida
print("="*60)
print("  Setup completo - Todos los imports cargados")
print("  Modulos: dataclasses, logging, typing, abc, time, json")
print("="*60)

---

## Seccion 1: Clases, Estado y Comportamiento

Una **clase** es la unidad fundamental de OOP. Combina **estado** (datos) y **comportamiento** (operaciones sobre esos datos) en una sola entidad.

```
+-----------------------------------------+
|          CLASE: ModelConfig              |
+-----------------------------------------+
|  ESTADO (atributos)                     |
|    - model_name: str                    |
|    - temperature: float                 |
|    - max_tokens: int                    |
+-----------------------------------------+
|  COMPORTAMIENTO (metodos)               |
|    + __init__(...)     -> constructor    |
|    + __repr__(...)     -> representacion |
|    + validate()        -> validar config |
|    + to_dict()         -> exportar       |
+-----------------------------------------+
              |
              |  instanciar
              v
    config_gpt = ModelConfig("gpt-4", 0.7, 1024)
    config_claude = ModelConfig("claude", 0.3, 2048)
    (cada una es un OBJETO con su propio estado)
```

### Conceptos clave

| Concepto | Definicion | Analogia en AI/ML |
|---|---|---|
| **Clase** | Blueprint / plantilla | Arquitectura de un modelo (ej: Transformer) |
| **Instancia (objeto)** | Un ejemplar concreto | Un modelo entrenado con pesos especificos |
| **Atributo** | Variable ligada al objeto | Pesos, hiperparametros, configuracion |
| **Metodo** | Funcion ligada al objeto | `predict()`, `fit()`, `evaluate()` |
| **`__init__`** | Constructor, inicializa estado | Cargar pesos y configuracion al crear el modelo |
| **`self`** | Referencia a la instancia actual | "Este modelo en particular" |

> **Regla fundamental**: `self` es siempre el primer parametro de los metodos de instancia. Python lo pasa automaticamente cuando llamas `objeto.metodo()`.

In [ ]:
# =============================================================================
# Seccion 1: Clases basicas en contexto AI/ML
# =============================================================================

class ModelConfig:
    """Configuracion para un modelo de lenguaje."""
    
    def __init__(self, model_name: str, temperature: float, max_tokens: int):
        self.model_name = model_name
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.created_at = time.time()  # estado interno automatico
    
    def validate(self) -> bool:
        """Valida que la configuracion sea coherente."""
        if not (0.0 <= self.temperature <= 2.0):
            print(f"  ERROR: temperature={self.temperature} fuera de rango [0.0, 2.0]")
            return False
        if self.max_tokens <= 0:
            print(f"  ERROR: max_tokens={self.max_tokens} debe ser positivo")
            return False
        print(f"  OK: Configuracion valida para '{self.model_name}'")
        return True
    
    def to_dict(self) -> dict:
        """Exporta la configuracion como diccionario."""
        return {
            "model_name": self.model_name,
            "temperature": self.temperature,
            "max_tokens": self.max_tokens,
        }
    
    def __repr__(self) -> str:
        return (f"ModelConfig(model='{self.model_name}', "
                f"temp={self.temperature}, tokens={self.max_tokens})")


class Prediction:
    """Resultado de una prediccion de modelo."""
    
    def __init__(self, label: str, score: float):
        self.label = label
        self.score = score
        self.timestamp = time.time()
    
    def is_confident(self, threshold: float = 0.8) -> bool:
        """Determina si la prediccion supera el umbral de confianza."""
        return self.score >= threshold
    
    def __repr__(self) -> str:
        return f"Prediction(label='{self.label}', score={self.score:.3f})"


# --- Uso ---
print("=== Creando configuraciones ===")
config_gpt = ModelConfig("gpt-4o", 0.7, 1024)
config_claude = ModelConfig("claude-opus", 0.3, 4096)

print(config_gpt)
print(config_claude)
print()

print("=== Validando ===")
config_gpt.validate()
config_malo = ModelConfig("test", 5.0, -1)  # config invalida
config_malo.validate()
print()

print("=== Predicciones ===")
pred1 = Prediction("positivo", 0.95)
pred2 = Prediction("negativo", 0.42)

print(f"{pred1} -> confiable: {pred1.is_confident()}")
print(f"{pred2} -> confiable: {pred2.is_confident()}")
print()

print("=== Exportar a dict ===")
print(json.dumps(config_gpt.to_dict(), indent=2))

---

## Seccion 2: Encapsulamiento - Proteger Invariantes

El **encapsulamiento** es el principio de ocultar los detalles internos y exponer solo lo necesario. En Python usamos convenciones:

```
Nivel de acceso en Python:

  self.publico        -> acceso libre desde cualquier lugar
  self._protegido     -> convencion: "no toques esto desde afuera"
  self.__privado      -> name mangling (Python renombra a _Clase__privado)

  Patron recomendado:
  +--------------------------------------------------+
  |  Atributo interno: self._valor                   |
  |                                                  |
  |  @property          -> lectura controlada        |
  |  def valor(self):                                |
  |      return self._valor                          |
  |                                                  |
  |  @valor.setter      -> escritura con validacion  |
  |  def valor(self, nuevo):                         |
  |      if es_valido(nuevo):                        |
  |          self._valor = nuevo                     |
  |      else:                                       |
  |          raise ValueError(...)                   |
  +--------------------------------------------------+
```

### Por que encapsular?

En AI/ML hay muchos valores que **no pueden ser cualquier cosa**:

| Invariante | Restriccion | Sin encapsulamiento |
|---|---|---|
| Score de confianza | 0.0 <= score <= 1.0 | Alguien pone `score = 5.0` y rompe todo |
| Temperature de LLM | 0.0 <= temp <= 2.0 | `temp = -1` genera error en la API |
| Nombre de modelo | No puede estar vacio | `name = ""` causa bugs silenciosos |
| Conteo de tokens | Entero positivo | `tokens = -500` es absurdo |

In [ ]:
# =============================================================================
# Seccion 2: Encapsulamiento con @property
# =============================================================================

class ConfidenceScore:
    """Score de confianza que SIEMPRE esta en el rango [0.0, 1.0]."""
    
    def __init__(self, value: float):
        # Usamos el setter para validar incluso en __init__
        self.value = value  # esto llama al @value.setter
    
    @property
    def value(self) -> float:
        """Lectura del score."""
        return self._value
    
    @value.setter
    def value(self, new_value: float):
        """Escritura con validacion estricta."""
        if not isinstance(new_value, (int, float)):
            raise TypeError(f"Score debe ser numerico, recibido: {type(new_value).__name__}")
        if not (0.0 <= new_value <= 1.0):
            raise ValueError(f"Score debe estar en [0.0, 1.0], recibido: {new_value}")
        self._value = float(new_value)
    
    def __repr__(self) -> str:
        return f"ConfidenceScore({self._value:.4f})"


class ModelRegistry:
    """Registro de modelos que previene duplicados y expone conteo read-only."""
    
    def __init__(self):
        self._models: Dict[str, dict] = {}  # interno, no acceder directamente
    
    @property
    def count(self) -> int:
        """Numero de modelos registrados (solo lectura)."""
        return len(self._models)
    
    @property
    def model_names(self) -> List[str]:
        """Lista de nombres registrados (copia, no referencia interna)."""
        return list(self._models.keys())
    
    def register(self, name: str, metadata: dict) -> None:
        """Registra un modelo. Lanza error si ya existe."""
        if not name or not name.strip():
            raise ValueError("El nombre del modelo no puede estar vacio")
        if name in self._models:
            raise ValueError(f"Modelo '{name}' ya esta registrado")
        self._models[name] = {"metadata": metadata, "registered_at": time.time()}
    
    def get(self, name: str) -> Optional[dict]:
        """Obtiene metadata de un modelo."""
        return self._models.get(name)
    
    def __repr__(self) -> str:
        return f"ModelRegistry(count={self.count}, models={self.model_names})"


# --- Demostracion: ConfidenceScore ---
print("=== ConfidenceScore ===")
score = ConfidenceScore(0.87)
print(f"Creado: {score}")
print(f"Acceso via property: {score.value}")

score.value = 0.95  # actualizacion valida
print(f"Actualizado: {score}")

print("\nIntentando valores invalidos:")
for val_invalido in [1.5, -0.3, "hola"]:
    try:
        score.value = val_invalido
    except (ValueError, TypeError) as e:
        print(f"  Rechazado {val_invalido!r}: {e}")

print(f"\nValor final intacto: {score}")

# --- Demostracion: ModelRegistry ---
print("\n=== ModelRegistry ===")
registry = ModelRegistry()
registry.register("gpt-4o", {"provider": "openai", "type": "chat"})
registry.register("claude-opus", {"provider": "anthropic", "type": "chat"})
print(registry)
print(f"Conteo (read-only): {registry.count}")

print("\nIntentando duplicado:")
try:
    registry.register("gpt-4o", {"provider": "openai"})
except ValueError as e:
    print(f"  Rechazado: {e}")

---

## Seccion 3: Herencia y Polimorfismo

La **herencia** permite crear clases que comparten interfaz y comportamiento. El **polimorfismo** permite tratar objetos diferentes de forma uniforme.

```
         +--------------------+
         |   BasePredictor    |  <- clase abstracta (ABC)
         |   (interfaz comun) |
         +---------+----------+
                   |
          +--------+--------+
          |                 |
  +-------+------+  +------+--------+
  | RuleBased    |  | Threshold     |
  | Predictor    |  | Predictor     |
  +--------------+  +---------------+
  | predict()    |  | predict()     |
  | (reglas)     |  | (umbral)      |
  +--------------+  +---------------+

  Polimorfismo: cualquier funcion que reciba un BasePredictor
  funciona con AMBAS subclases sin saber cual es.
```

### Cuando usar (y cuando NO usar) herencia

| Situacion | Usar herencia? | Motivo |
|---|---|---|
| Varios predictores con misma interfaz `predict()` | **Si** | Relacion "es-un" genuina |
| Todos los modelos necesitan `save()` y `load()` | **Si** | Interfaz comun, implementacion diferente |
| Quiero reusar el metodo `log_request()` de otra clase | **No** | Eso es composicion, no herencia |
| Tengo 4+ niveles de herencia | **No** | Jerarquias profundas son fragiles |
| La subclase usa solo el 30% de la clase padre | **No** | Herencia excesiva, mejor componer |

In [ ]:
# =============================================================================
# Seccion 3: Herencia, ABC y polimorfismo
# =============================================================================

class BasePredictor(ABC):
    """Interfaz abstracta para cualquier predictor."""
    
    def __init__(self, name: str):
        self.name = name
    
    @abstractmethod
    def predict(self, text: str) -> Prediction:
        """Realiza una prediccion. Cada subclase lo implementa distinto."""
        pass
    
    def __repr__(self) -> str:
        return f"{type(self).__name__}(name='{self.name}')"


class RuleBasedPredictor(BasePredictor):
    """Predictor basado en reglas de palabras clave."""
    
    def __init__(self, name: str, positive_words: List[str], negative_words: List[str]):
        super().__init__(name)
        self.positive_words = [w.lower() for w in positive_words]
        self.negative_words = [w.lower() for w in negative_words]
    
    def predict(self, text: str) -> Prediction:
        text_lower = text.lower()
        pos_count = sum(1 for w in self.positive_words if w in text_lower)
        neg_count = sum(1 for w in self.negative_words if w in text_lower)
        total = pos_count + neg_count
        if total == 0:
            return Prediction("neutral", 0.5)
        score = pos_count / total
        label = "positivo" if score > 0.5 else "negativo" if score < 0.5 else "neutral"
        return Prediction(label, score)


class ThresholdPredictor(BasePredictor):
    """Predictor que clasifica basado en longitud de texto y umbral."""
    
    def __init__(self, name: str, threshold: int = 50):
        super().__init__(name)
        self.threshold = threshold
    
    def predict(self, text: str) -> Prediction:
        length = len(text)
        if length > self.threshold:
            score = min(length / (self.threshold * 3), 1.0)
            return Prediction("detallado", score)
        else:
            score = max(1.0 - (length / self.threshold), 0.0)
            return Prediction("breve", score)


# --- Polimorfismo en accion ---
def evaluate_predictor(predictor: BasePredictor, texts: List[str]) -> None:
    """Funciona con CUALQUIER subclase de BasePredictor."""
    print(f"\nEvaluando: {predictor}")
    print("-" * 50)
    for text in texts:
        result = predictor.predict(text)
        snippet = text[:40] + "..." if len(text) > 40 else text
        print(f"  '{snippet}' -> {result}")


# Crear predictores diferentes
rule_pred = RuleBasedPredictor(
    "sentimiento-reglas",
    positive_words=["bueno", "excelente", "genial", "rapido"],
    negative_words=["malo", "lento", "error", "falla"]
)

threshold_pred = ThresholdPredictor("longitud-texto", threshold=30)

# Textos de prueba
textos = [
    "El modelo es bueno y rapido",
    "Hay un error malo en la falla del sistema",
    "Hola",
    "Este es un texto mucho mas largo que deberia clasificarse como detallado por el predictor",
]

# La MISMA funcion funciona con ambos predictores (polimorfismo)
for predictor in [rule_pred, threshold_pred]:
    evaluate_predictor(predictor, textos)

# --- No puedes instanciar la clase abstracta ---
print("\n=== Intentando instanciar ABC ===")
try:
    base = BasePredictor("base")
except TypeError as e:
    print(f"  Error esperado: {e}")

---

## Seccion 4: Composicion sobre Herencia

El principio **"prefiere composicion sobre herencia"** es una de las guias mas importantes en diseno orientado a objetos.

```
  HERENCIA (is-a)                COMPOSICION (has-a)
  ================               ==================

  InferenceService               InferenceService
       |                          |-- preprocessor: Preprocessor
       v                          |-- model: Model
  PreprocessingService            |-- postprocessor: Postprocessor
       |                          (cada pieza es independiente
       v                           y se puede reemplazar)
  ModelService
       |
       v
  PostprocessingService
  (cadena rigida, dificil
   de modificar o testear)
```

### Comparacion directa

| Aspecto | Herencia | Composicion |
|---|---|---|
| **Relacion** | "Es un" (is-a) | "Tiene un" (has-a) |
| **Acoplamiento** | Alto (subclase depende de padre) | Bajo (piezas independientes) |
| **Flexibilidad** | Rigida (jerarquia fija) | Flexible (intercambiar piezas) |
| **Testing** | Dificil aislar partes | Facil testear cada pieza |
| **Reutilizacion** | Solo via jerarquia | En cualquier combinacion |
| **Cuando usar** | Interfaz comun genuina | La mayoria de los demas casos |

> **Regla practica**: Si dudas entre herencia y composicion, elige composicion. Solo usa herencia cuando la relacion "es-un" es clara e inmutable.

In [ ]:
# =============================================================================
# Seccion 4: Composicion en un servicio de inferencia
# =============================================================================

# --- Componentes independientes ---

class Preprocessor:
    """Limpia y normaliza el texto de entrada."""
    
    def process(self, text: str) -> str:
        cleaned = text.strip().lower()
        print(f"    [Preprocessor] '{text[:30]}...' -> '{cleaned[:30]}...'")
        return cleaned


class FakeModel:
    """Simula un modelo de ML (sin dependencias externas)."""
    
    def __init__(self, name: str):
        self.name = name
    
    def predict(self, text: str) -> dict:
        # Simulacion: score basado en longitud
        score = min(len(text) / 100, 1.0)
        result = {"label": "positivo" if score > 0.5 else "negativo", "score": score}
        print(f"    [FakeModel:{self.name}] score={score:.3f}")
        return result


class Postprocessor:
    """Formatea y enriquece la salida del modelo."""
    
    def __init__(self, confidence_threshold: float = 0.7):
        self.confidence_threshold = confidence_threshold
    
    def process(self, raw_result: dict) -> dict:
        enriched = {
            **raw_result,
            "is_confident": raw_result["score"] >= self.confidence_threshold,
            "timestamp": time.time(),
        }
        print(f"    [Postprocessor] confident={enriched['is_confident']}")
        return enriched


# --- Servicio compuesto ---

class InferenceService:
    """Servicio de inferencia que COMPONE (has-a) sus partes.
    
    Cada componente es independiente y se puede reemplazar.
    """
    
    def __init__(self, preprocessor: Preprocessor, model: FakeModel, postprocessor: Postprocessor):
        self.preprocessor = preprocessor
        self.model = model
        self.postprocessor = postprocessor
    
    def run(self, text: str) -> dict:
        """Ejecuta el pipeline completo."""
        print(f"  Pipeline de inferencia iniciado")
        cleaned = self.preprocessor.process(text)
        raw = self.model.predict(cleaned)
        final = self.postprocessor.process(raw)
        print(f"  Pipeline completado")
        return final


# --- Uso: Composicion permite intercambiar piezas ---
print("=== Servicio con modelo A ===")
service_a = InferenceService(
    preprocessor=Preprocessor(),
    model=FakeModel("modelo-sentimiento-v1"),
    postprocessor=Postprocessor(confidence_threshold=0.7)
)
result_a = service_a.run("  Este producto es EXCELENTE y lo recomiendo totalmente para todos  ")
print(f"  Resultado: {json.dumps(result_a, indent=2, default=str)}")

print("\n=== Mismo servicio, modelo diferente (swap facil!) ===")
service_b = InferenceService(
    preprocessor=Preprocessor(),
    model=FakeModel("modelo-sentimiento-v2"),  # <- solo cambiamos el modelo
    postprocessor=Postprocessor(confidence_threshold=0.5)  # <- y el umbral
)
result_b = service_b.run("  Malo  ")
print(f"  Resultado: {json.dumps(result_b, indent=2, default=str)}")

print("\n--- Ventaja clave ---")
print("Con composicion, cambiar un componente NO requiere modificar los demas.")
print("Con herencia profunda, un cambio en la clase padre puede romper TODAS las subclases.")

---

## Seccion 5: Dataclasses - Datos Estructurados sin Boilerplate

Las **dataclasses** (Python 3.7+) generan automaticamente `__init__`, `__repr__`, `__eq__` y mas. Ideales para objetos que son principalmente **contenedores de datos**.

### Clase regular vs Dataclass

```python
# --- SIN dataclass: mucho codigo repetitivo ---
class ExperimentResult:
    def __init__(self, model_name, accuracy, params):
        self.model_name = model_name
        self.accuracy = accuracy
        self.params = params
    
    def __repr__(self):
        return (f"ExperimentResult(model_name={self.model_name!r}, "
                f"accuracy={self.accuracy}, params={self.params})")
    
    def __eq__(self, other):
        return (self.model_name == other.model_name 
                and self.accuracy == other.accuracy
                and self.params == other.params)

# --- CON dataclass: Python genera todo lo anterior ---
@dataclass
class ExperimentResult:
    model_name: str
    accuracy: float
    params: dict = field(default_factory=dict)
```

### Opciones importantes

| Opcion | Default | Efecto |
|---|---|---|
| `@dataclass` | - | Genera `__init__`, `__repr__`, `__eq__` |
| `frozen=True` | False | Instancias inmutables (no puedes cambiar atributos) |
| `order=True` | False | Genera `__lt__`, `__le__`, etc. para ordenar |
| `field(default_factory=list)` | - | Default seguro para tipos mutables |

In [ ]:
# =============================================================================
# Seccion 5: Dataclasses en practica
# =============================================================================

@dataclass
class ExperimentResult:
    """Resultado de un experimento de ML."""
    model_name: str
    accuracy: float
    f1_score: float
    params: dict = field(default_factory=dict)
    tags: List[str] = field(default_factory=list)


@dataclass(frozen=True)
class ModelVersion:
    """Version inmutable de un modelo (no se puede modificar despues de creada)."""
    name: str
    version: str
    checksum: str


@dataclass(order=True)
class RankedModel:
    """Modelo con ranking. Ordena por score automaticamente."""
    score: float  # campo de ordenamiento (primer campo)
    name: str = field(compare=False)  # no usar para comparar
    details: str = field(compare=False, repr=False)


# --- __repr__ automatico ---
print("=== __repr__ automatico ===")
exp1 = ExperimentResult("bert-base", 0.89, 0.87, {"lr": 0.001}, ["nlp", "produccion"])
exp2 = ExperimentResult("gpt-finetune", 0.92, 0.91, {"lr": 0.0001})
print(exp1)
print(exp2)

# --- __eq__ automatico ---
print("\n=== __eq__ automatico ===")
exp3 = ExperimentResult("bert-base", 0.89, 0.87, {"lr": 0.001}, ["nlp", "produccion"])
print(f"exp1 == exp3: {exp1 == exp3}")  # True, mismos valores
print(f"exp1 == exp2: {exp1 == exp2}")  # False, valores distintos

# --- frozen=True: inmutabilidad ---
print("\n=== frozen=True (inmutable) ===")
version = ModelVersion("sentiment-model", "1.2.0", "abc123hash")
print(version)
try:
    version.name = "otro-nombre"  # no se puede!
except AttributeError as e:
    print(f"  Inmutable! Error: {e}")

# --- order=True: ordenamiento automatico ---
print("\n=== order=True (ordenamiento) ===")
models = [
    RankedModel(0.85, "modelo-A", "baseline"),
    RankedModel(0.92, "modelo-B", "mejorado"),
    RankedModel(0.78, "modelo-C", "experimental"),
    RankedModel(0.95, "modelo-D", "champion"),
]
for m in sorted(models, reverse=True):
    print(f"  {m.score:.2f} - {m.name}")

# --- field(default_factory=...) evita el bug de mutable default ---
print("\n=== default_factory (seguridad con mutables) ===")
a = ExperimentResult("modelo-a", 0.9, 0.88)
b = ExperimentResult("modelo-b", 0.8, 0.75)
a.tags.append("produccion")
print(f"Tags de a: {a.tags}")
print(f"Tags de b: {b.tags}")  # NO contaminado, cada uno tiene su propia lista

---

## Seccion 6: Logging en Clases

El **logging** es esencial en produccion para rastrear el comportamiento de tus pipelines de ML. Cada clase deberia tener su propio logger.

### Patron recomendado

```python
class MiClase:
    def __init__(self):
        self._logger = logging.getLogger(type(self).__name__)
        self._logger.info("Instancia creada")  # ciclo de vida
    
    def hacer_algo(self):
        self._logger.debug("Procesando...")    # detalle
        self._logger.warning("Dato sospechoso") # advertencia
        self._logger.error("Algo fallo")        # error
```

### Niveles de logging

```
  DEBUG    -> Detalles de ejecucion paso a paso
  INFO     -> Eventos normales del ciclo de vida
  WARNING  -> Algo inesperado pero no fatal
  ERROR    -> Algo fallo, la operacion no se completo
  CRITICAL -> Falla total del sistema
```

> **Buena practica**: Usa `type(self).__name__` en vez de `__name__` para que las subclases automaticamente obtengan su propio nombre de logger.

In [ ]:
# =============================================================================
# Seccion 6: Logging en clases de ML
# =============================================================================

class MLPipeline:
    """Pipeline de ML con logging completo del ciclo de vida."""
    
    def __init__(self, name: str, steps: List[str]):
        self._logger = logging.getLogger(type(self).__name__)
        self.name = name
        self.steps = steps
        self._logger.info(f"Pipeline '{name}' creado con {len(steps)} pasos: {steps}")
    
    def process(self, data: List[str]) -> List[dict]:
        """Procesa una lista de textos a traves del pipeline."""
        self._logger.info(f"Iniciando procesamiento de {len(data)} elementos")
        results = []
        
        for i, item in enumerate(data):
            self._logger.debug(f"Procesando item {i+1}/{len(data)}: '{item[:30]}...'")
            
            # Advertencia si el texto es muy corto
            if len(item.strip()) < 5:
                self._logger.warning(f"Item {i+1} tiene texto muy corto ({len(item)} chars), "
                                     f"resultados pueden ser poco confiables")
            
            # Simular procesamiento
            try:
                if not item.strip():
                    raise ValueError("Texto vacio no permitido")
                result = {
                    "input": item,
                    "output": f"procesado_{item[:20].lower()}",
                    "steps_applied": self.steps,
                }
                results.append(result)
            except ValueError as e:
                self._logger.error(f"Error en item {i+1}: {e}")
        
        self._logger.info(f"Procesamiento completado: {len(results)}/{len(data)} exitosos")
        return results


# --- Uso ---
print("=" * 60)
print("  Observa los mensajes de logging (nivel, clase, mensaje)")
print("=" * 60)
print()

pipeline = MLPipeline(
    name="sentimiento-pipeline",
    steps=["tokenize", "normalize", "predict"]
)

data = [
    "Este producto es excelente, lo recomiendo",
    "Hola",       # texto corto -> WARNING
    "",           # texto vacio -> ERROR
    "El servicio al cliente fue pesimo y tardaron mucho",
]

results = pipeline.process(data)
print(f"\nResultados exitosos: {len(results)}")

---

## Seccion 7: Criterios de Diseno

No todo necesita ser una clase. Saber **cuando usar cada herramienta** es la clave del buen diseno.

### Tabla de decision

| Situacion | Herramienta | Motivo |
|---|---|---|
| Transformar A -> B sin estado | **Funcion** | Simple, testeable, sin complejidad innecesaria |
| Agrupar datos relacionados | **Dataclass** | Estructura clara, metodos auto-generados |
| Mantener estado + comportamiento | **Clase** | Encapsulamiento, invariantes protegidos |
| Operacion unica sin memoria | **Funcion** | Una clase con un solo metodo = funcion disfrazada |
| Multiples implementaciones de una interfaz | **ABC + clases** | Polimorfismo genuino |
| Configuracion inmutable | **Dataclass(frozen=True)** | Seguridad, hasheable |

### Principios clave

```
1. RESPONSABILIDAD UNICA
   Una clase = una razon para cambiar.
   Si describes la clase con "Y", probablemente son dos clases.

2. SUPERFICIE MINIMA
   Exponer lo menos posible. Mas metodos publicos = mas contratos que mantener.

3. COMPORTAMIENTO SOBRE DATOS
   Si una clase solo tiene getters/setters sin logica, probablemente deberia
   ser un diccionario o una dataclass.

4. COMPOSICION SOBRE HERENCIA
   Componer piezas es mas flexible que heredar cadenas.
```

In [ ]:
# =============================================================================
# Seccion 7: Tres enfoques para el mismo problema
# =============================================================================
# Problema: wrapper para un proveedor de LLM con reintentos y cache

# -----------------------------------------------------------------------
# ENFOQUE 1: Solo funciones (procedural)
# Ventaja: simple, sin estado oculto
# Desventaja: cada llamada necesita pasar toda la configuracion
# -----------------------------------------------------------------------

def call_llm_functional(
    prompt: str,
    model: str,
    temperature: float,
    max_retries: int,
    cache: dict,  # estado externo que el caller debe manejar
) -> str:
    """Enfoque funcional: todo se pasa como parametro."""
    if prompt in cache:
        return cache[prompt]
    # Simular llamada
    response = f"[{model}] Respuesta para: {prompt[:30]}..."
    cache[prompt] = response
    return response


# -----------------------------------------------------------------------
# ENFOQUE 2: Clase con estado (OOP completo)
# Ventaja: estado encapsulado, API limpia
# Desventaja: mas codigo, mas complejidad
# -----------------------------------------------------------------------

class LLMClient:
    """Enfoque OOP: estado encapsulado, metodos claros."""
    
    def __init__(self, model: str, temperature: float = 0.7, max_retries: int = 3):
        self._model = model
        self._temperature = temperature
        self._max_retries = max_retries
        self._cache: Dict[str, str] = {}
        self._call_count = 0
    
    def call(self, prompt: str) -> str:
        if prompt in self._cache:
            return self._cache[prompt]
        self._call_count += 1
        response = f"[{self._model}] Respuesta para: {prompt[:30]}..."
        self._cache[prompt] = response
        return response
    
    @property
    def stats(self) -> dict:
        return {"calls": self._call_count, "cache_size": len(self._cache)}


# -----------------------------------------------------------------------
# ENFOQUE 3: Dataclass para config + funciones para comportamiento
# Ventaja: datos y logica separados, muy testeable
# Desventaja: no hay encapsulamiento del estado mutable
# -----------------------------------------------------------------------

@dataclass(frozen=True)
class LLMConfig:
    """Configuracion inmutable del LLM."""
    model: str
    temperature: float = 0.7
    max_retries: int = 3


def call_llm_with_config(config: LLMConfig, prompt: str, cache: dict) -> str:
    """Funcion pura (casi) que usa configuracion inmutable."""
    if prompt in cache:
        return cache[prompt]
    response = f"[{config.model}] Respuesta para: {prompt[:30]}..."
    cache[prompt] = response
    return response


# --- Comparacion ---
print("=" * 60)
print("  COMPARACION: 3 enfoques para wrapper de LLM")
print("=" * 60)

prompt = "Explica que es machine learning en 2 oraciones"

# Enfoque 1: Funcional
cache1: dict = {}
r1 = call_llm_functional(prompt, "gpt-4", 0.7, 3, cache1)
print(f"\n1) Funcional:      {r1}")

# Enfoque 2: Clase
client = LLMClient("gpt-4")
r2 = client.call(prompt)
print(f"2) Clase:          {r2}")
print(f"   Stats:          {client.stats}")

# Enfoque 3: Dataclass + funcion
config = LLMConfig("gpt-4")
cache3: dict = {}
r3 = call_llm_with_config(config, prompt, cache3)
print(f"3) Dataclass+func: {r3}")

print("\n" + "-" * 60)
print("Recomendacion:")
print("  - Si hay estado mutable importante (cache, conteo) -> Clase (enfoque 2)")
print("  - Si la config es fija y la logica es simple       -> Dataclass + funcion (enfoque 3)")
print("  - Si es una operacion suelta sin estado            -> Funcion pura (enfoque 1)")

---

### Recapitulacion rapida antes de los ejercicios

Hasta aqui hemos cubierto **7 secciones** con los pilares de OOP aplicada:

```
  Seccion 1: Clases          ->  __init__, self, atributos, metodos
  Seccion 2: Encapsulamiento ->  @property, validacion, invariantes
  Seccion 3: Herencia        ->  ABC, @abstractmethod, polimorfismo
  Seccion 4: Composicion     ->  has-a, piezas intercambiables
  Seccion 5: Dataclasses     ->  @dataclass, field(), frozen
  Seccion 6: Logging         ->  logging.getLogger, niveles
  Seccion 7: Criterios       ->  funcion vs dataclass vs clase
```

Los ejercicios que siguen integran **multiples secciones** en cada problema. Si necesitas repasar, vuelve a la seccion correspondiente.

---

## Ejercicios

Pon en practica todo lo visto. Cada ejercicio integra multiples conceptos de las secciones anteriores.

> **Nota**: Los cuerpos de los metodos estan con `pass`. Reemplaza `pass` con tu implementacion.

In [ ]:
# =============================================================================
# EJERCICIO 1: LLMProvider con composicion
# =============================================================================
#
# Conceptos: composicion, encapsulamiento, logging
#
# Instrucciones:
#   1. Completa la clase RateLimiter:
#      - Lleva un conteo de llamadas (_call_count) y un limite (max_calls)
#      - El metodo check() retorna True si hay llamadas disponibles,
#        False si se alcanzo el limite
#      - El metodo record_call() incrementa el conteo
#
#   2. Completa la clase ResponseCache:
#      - Usa un dict interno para almacenar respuestas
#      - get(key) retorna la respuesta cacheada o None
#      - set(key, value) almacena la respuesta
#
#   3. Completa la clase LLMProvider:
#      - COMPONE un RateLimiter y un ResponseCache
#      - Tiene su propio logger (logging.getLogger)
#      - El metodo generate(prompt):
#        a) Revisa el cache -> si hay respuesta, retornarla (log DEBUG)
#        b) Revisa rate limit -> si no hay cuota, lanzar RuntimeError (log WARNING)
#        c) "Llama a la API" (simula con f"Respuesta para: {prompt}")
#        d) Cachea el resultado y registra la llamada (log INFO)
#        e) Retorna la respuesta
#
# Pista: Mira la Seccion 4 (composicion) y Seccion 6 (logging) como referencia.
# =============================================================================

class RateLimiter:
    """Limita la cantidad de llamadas permitidas."""
    
    def __init__(self, max_calls: int):
        self._max_calls = max_calls
        self._call_count = 0
    
    def check(self) -> bool:
        """Retorna True si hay cuota disponible."""
        pass
    
    def record_call(self) -> None:
        """Registra una llamada realizada."""
        pass
    
    @property
    def remaining(self) -> int:
        """Llamadas restantes."""
        pass


class ResponseCache:
    """Cache simple de respuestas."""
    
    def __init__(self):
        self._store: Dict[str, str] = {}
    
    def get(self, key: str) -> Optional[str]:
        """Retorna respuesta cacheada o None."""
        pass
    
    def set(self, key: str, value: str) -> None:
        """Almacena una respuesta."""
        pass
    
    @property
    def size(self) -> int:
        """Numero de entradas en cache."""
        pass


class LLMProvider:
    """Proveedor de LLM que compone RateLimiter y ResponseCache."""
    
    def __init__(self, model_name: str, max_calls: int = 10):
        self.model_name = model_name
        self._rate_limiter = RateLimiter(max_calls)
        self._cache = ResponseCache()
        self._logger = logging.getLogger(type(self).__name__)
        self._logger.info(f"Proveedor '{model_name}' inicializado (limite: {max_calls} llamadas)")
    
    def generate(self, prompt: str) -> str:
        """Genera respuesta: cache -> rate limit -> API simulada -> cache."""
        pass


# --- Prueba tu implementacion ---
# Descomenta las siguientes lineas cuando completes el ejercicio:
#
# provider = LLMProvider("gpt-4o", max_calls=3)
# print(provider.generate("Hola mundo"))           # llamada API
# print(provider.generate("Hola mundo"))           # desde cache
# print(provider.generate("Otra pregunta"))        # llamada API
# print(provider.generate("Tercera pregunta"))     # llamada API
# print(provider.generate("Cuarta pregunta"))      # RuntimeError: rate limit

In [ ]:
# =============================================================================
# EJERCICIO 2: ModelExperimentTracker con dataclasses
# =============================================================================
#
# Conceptos: dataclasses, field(), metodos sobre colecciones
#
# Instrucciones:
#   1. La dataclass Experiment ya esta definida. NO la modifiques.
#
#   2. Completa ExperimentTracker:
#      - add_experiment(exp): agrega un experimento a la lista interna
#      - best_by_metric(metric): retorna el experimento con el valor mas alto
#        para esa metrica. Si ninguno tiene esa metrica, retorna None.
#        (Las metricas estan en exp.metrics, que es un dict)
#      - filter_by_tag(tag): retorna una lista de experimentos que contengan
#        ese tag en su lista de tags.
#      - summary(): retorna un dict con:
#        {"total": N, "models": [lista de nombres unicos], "all_tags": [tags unicos]}
#
# Pista: Usa max() con key= para encontrar el mejor por metrica.
#        Usa set comprehensions para tags/nombres unicos.
# =============================================================================

@dataclass
class Experiment:
    """Un experimento individual."""
    name: str
    model_name: str
    metrics: Dict[str, float] = field(default_factory=dict)
    tags: List[str] = field(default_factory=list)


class ExperimentTracker:
    """Tracker que almacena y consulta experimentos."""
    
    def __init__(self):
        self._experiments: List[Experiment] = []
    
    def add_experiment(self, exp: Experiment) -> None:
        """Agrega un experimento al tracker."""
        pass
    
    def best_by_metric(self, metric: str) -> Optional[Experiment]:
        """Retorna el experimento con el valor mas alto para la metrica dada."""
        pass
    
    def filter_by_tag(self, tag: str) -> List[Experiment]:
        """Retorna experimentos que tengan el tag especificado."""
        pass
    
    def summary(self) -> dict:
        """Resumen general del tracker."""
        pass


# --- Prueba tu implementacion ---
# Descomenta las siguientes lineas cuando completes el ejercicio:
#
# tracker = ExperimentTracker()
# tracker.add_experiment(Experiment("exp-001", "bert", {"accuracy": 0.89, "f1": 0.87}, ["nlp", "v1"]))
# tracker.add_experiment(Experiment("exp-002", "gpt", {"accuracy": 0.92, "f1": 0.90}, ["nlp", "v2"]))
# tracker.add_experiment(Experiment("exp-003", "bert", {"accuracy": 0.91, "f1": 0.89}, ["nlp", "v2", "produccion"]))
#
# print("Mejor por accuracy:", tracker.best_by_metric("accuracy"))
# print("Mejor por f1:", tracker.best_by_metric("f1"))
# print("Tag 'v2':", tracker.filter_by_tag("v2"))
# print("Summary:", tracker.summary())

In [ ]:
# =============================================================================
# EJERCICIO 4 (BONUS): Verificacion rapida de conceptos
# =============================================================================
# Ejecuta esta celda para verificar que entiendes los conceptos.
# Cada assert debe pasar sin error. Si alguno falla, revisa la seccion
# correspondiente.
#
# NO necesitas modificar nada. Solo ejecuta y observa.
# =============================================================================

# --- Test 1: Clases basicas ---
class _TestModel:
    def __init__(self, name):
        self.name = name
    def greet(self):
        return f"Soy {self.name}"

_m = _TestModel("test")
assert _m.name == "test", "Seccion 1: atributos de instancia"
assert _m.greet() == "Soy test", "Seccion 1: metodos"
print("Seccion 1 (Clases):          OK")

# --- Test 2: Encapsulamiento ---
_cs = ConfidenceScore(0.5)
_error_raised = False
try:
    _cs.value = 2.0
except ValueError:
    _error_raised = True
assert _error_raised, "Seccion 2: @property setter debe rechazar valores invalidos"
assert _cs.value == 0.5, "Seccion 2: valor no debe cambiar tras rechazo"
print("Seccion 2 (Encapsulamiento):  OK")

# --- Test 3: Herencia ---
assert isinstance(rule_pred, BasePredictor), "Seccion 3: subclase es instancia del padre"
_pred_result = rule_pred.predict("bueno")
assert isinstance(_pred_result, Prediction), "Seccion 3: predict retorna Prediction"
print("Seccion 3 (Herencia):         OK")

# --- Test 4: Dataclasses ---
_dc1 = ExperimentResult("a", 0.9, 0.8)
_dc2 = ExperimentResult("a", 0.9, 0.8)
assert _dc1 == _dc2, "Seccion 5: __eq__ auto-generado en dataclass"
_dc1.tags.append("x")
assert _dc2.tags == [], "Seccion 5: default_factory aisle listas"
print("Seccion 5 (Dataclasses):      OK")

# --- Test 5: frozen ---
_fv = ModelVersion("m", "1.0", "hash")
_frozen_error = False
try:
    _fv.name = "otro"
except AttributeError:
    _frozen_error = True
assert _frozen_error, "Seccion 5: frozen=True impide modificacion"
print("Seccion 5 (frozen):           OK")

print("\n" + "=" * 50)
print("  Todos los conceptos verificados correctamente!")
print("=" * 50)

In [ ]:
# =============================================================================
# EJERCICIO 3: Refactorizar una clase con demasiadas responsabilidades
# =============================================================================
#
# Conceptos: responsabilidad unica, composicion, diseno limpio
#
# La siguiente clase "DoEverythingModel" viola el principio de responsabilidad
# unica. Tiene DEMASIADAS responsabilidades mezcladas.
#
# Tu tarea: Refactoriza en clases mas pequenas y componlas.
#
# Pistas:
#   - Identifica las responsabilidades (al menos 3 distintas)
#   - Crea una clase por responsabilidad
#   - Crea una clase orquestadora que las componga
#   - Cada clase debe poder testearse independientemente
# =============================================================================

# --- CLASE PROBLEMATICA (NO modificar, es el "antes") ---

class DoEverythingModel:
    """ANTI-PATRON: Esta clase hace demasiadas cosas."""
    
    def __init__(self, model_name: str, db_path: str, log_file: str):
        self.model_name = model_name
        self.db_path = db_path
        self.log_file = log_file
        self.predictions = []       # almacena predicciones (responsabilidad: storage)
        self.metrics = {}           # calcula metricas (responsabilidad: evaluacion)
        self.call_count = 0         # rate limiting (responsabilidad: rate limit)
        self.max_calls = 100
    
    def predict(self, text: str) -> dict:
        # Rate limiting (responsabilidad 1)
        if self.call_count >= self.max_calls:
            raise RuntimeError("Rate limit alcanzado")
        self.call_count += 1
        
        # Preprocesamiento (responsabilidad 2)
        text = text.strip().lower()
        
        # Prediccion (responsabilidad 3)
        result = {"label": "positivo", "score": 0.85}
        
        # Storage (responsabilidad 4)
        self.predictions.append({"input": text, "output": result})
        
        # Logging (responsabilidad 5)
        print(f"[LOG] {self.model_name} predijo {result['label']} para '{text[:20]}...'")
        
        # Metricas (responsabilidad 6)
        self.metrics["total_predictions"] = len(self.predictions)
        
        return result


# --- TU REFACTORIZACION (escribe aqui debajo) ---
# Crea las clases necesarias y luego una clase orquestadora
# que las componga para lograr el mismo resultado.

pass

---

## Consolidacion

### Checklist de aprendizaje

Marca cada item que puedas explicar con confianza:

- [ ] Se como definir una clase con `__init__`, atributos y metodos
- [ ] Entiendo que `self` es la referencia a la instancia actual
- [ ] Puedo proteger invariantes con `@property` y `@setter`
- [ ] Se la diferencia entre herencia (`is-a`) y composicion (`has-a`)
- [ ] Puedo crear clases abstractas con `ABC` y `@abstractmethod`
- [ ] Entiendo el polimorfismo: misma interfaz, distinta implementacion
- [ ] Se cuando usar `@dataclass` en vez de una clase regular
- [ ] Puedo usar `frozen=True` para crear objetos inmutables
- [ ] Se configurar logging por clase con `logging.getLogger`
- [ ] Puedo decidir entre funcion, dataclass y clase segun el problema

### Framework de decision: Herencia vs Composicion

```
Necesito compartir interfaz/contrato?
  |-- Si: Es una relacion "es-un" genuina?
  |     |-- Si: Las subclases usan >80% de la clase padre?
  |     |     |-- Si  -> HERENCIA (con ABC)
  |     |     |-- No  -> COMPOSICION
  |     |-- No -> COMPOSICION
  |-- No: Solo quiero reusar codigo?
        |-- Si -> COMPOSICION (o funciones)
        |-- No -> Probablemente no necesitas ninguna de las dos
```

### Resumen de secciones

| Seccion | Concepto clave | Patron principal |
|---|---|---|
| 1 | Clases y objetos | `class` + `__init__` + `self` |
| 2 | Encapsulamiento | `@property` + validacion en setter |
| 3 | Herencia/Polimorfismo | `ABC` + `@abstractmethod` |
| 4 | Composicion | `has-a` en vez de `is-a` |
| 5 | Dataclasses | `@dataclass` + `field()` |
| 6 | Logging | `logging.getLogger(type(self).__name__)` |
| 7 | Criterios de diseno | Funcion vs Dataclass vs Clase |

---

### Siguiente paso

**Notebook 05: Algoritmos y Estructuras de Datos** - donde aplicaremos OOP para implementar estructuras de datos comunes y algoritmos fundamentales en contextos de AI/ML.

---

*Notebook 04 - OOP Aplicada a AI/ML Engineering - Clase Extra de Python*